In [ ]:
# default_exp weahter

# Board API

> API details.

In [ ]:
#export

from datetime import datetime
import json
import logging
import requests

In [ ]:
#export 
IDENTITY_PATH = "/identity/connect/token"
QUERY_PATH = "/public/"

In [ ]:
#export 

logging.getLogger(__name__).addHandler(logging.NullHandler())

class Client():  # pylint: disable=too-few-public-methods
    """
    simple client to run queries and processes using the Board APIs.
    See related documentation when creating External Queries in a Board Datamodel
    
    """

    # initialize a new Client object - will retrieve and store bearer token
    def __init__(self, endpoint, client_id, client_secret, scope):
        self.endpoint = endpoint
        self.client_id = client_id
        self.client_secret = client_secret
        self.scope = scope

        # generate and store access token in self.token
        try:
            auth_url = self.endpoint + IDENTITY_PATH
            result = requests.post(
                auth_url,
                data={
                    'grant_type': 'client_credentials',
                    'scope': self.scope,
                    'client_id': self.client_id,
                    'client_secret': self.client_secret,
                }
            )
            self.token = json.loads(result.text)['access_token']
            logging.debug("Board API - Host: {} - Client ID: {} - token generated".format(self.endpoint,self.client_id))

        except Exception as error:
            logging.error("Board API - Host: {} - Client ID: {} - error generating token: {}".format(self.endpoint,self.client_id,str(error)))
            raise

    # call board api - GET request
    def __api_call_get(self,query):   
        """Run API Call - Get"""
        try:
            result = requests.get(
                self.endpoint + query,
                headers={
                    "Authorization":"Bearer " + self.token
                }
            )
            logging.debug("Board API - Host: {} - Client ID: {} - GET request: {} - executed succesfully".format(self.endpoint,self.client_id,query))
            return result.text

        except Exception as error:
            logging.error("Board API - Host: {} - Client ID: {} - GET request: {} - error: {}".format(self.endpoint,self.client_id,query,str(error)))
            raise
        pass

    # call board api - POST request
    def __api_call_post(self,query):   
        """Run API Call"""
        try:
            result = requests.post(
                self.endpoint + query,
                headers={
                    "Authorization":"Bearer " + self.token,
                    "Accept":"application/json"
                }
            )
            logging.debug("Board API - Host: {} - Client ID: {} - POST request: {} - executed succesfully".format(self.endpoint,self.client_id,query))
            return result.text

        except Exception as error:
            logging.error(
                "Board API\tHost: %s\tQuery: %s\tStatus: Error\tMessage: %s",
                self.endpoint, query, str(error)
            )
            raise
        pass




In [ ]:
#export

def query(self, datamodel: str, query_name: str, query_string= str):
    
        """
        Execute a datamodel external query and returns
        a python dictionary with the data from Board.

        The Function takes 3 Parameters: 
        
        -       datamodel(String) : Name of the Board Database 
        -       query_name(String) : Name of the Query 
        -       query_string(String) : Query in String Format
        
        
        """
        
        query = QUERY_PATH + "/" + datamodel + "/query/" + query_name + query_string
        
        return self.__api_call_get(query)

In [ ]:
#export 

def cubes(self, datamodel: str):

    """
    Return list of cubes for specified datamodel.

    The Function takes 1 Parameter:

    -   datamodel(String) : Name of the Board Database 
    
    """

    query = QUERY_PATH + "/" + datamodel + "/schema/Cubes"
    
    return self.__api_call_get(query)

In [ ]:
#export 

def entities(self,datamodel: str):
    
    """
    Return list of entities for specified datamodel.

    The Function takes 1 Parameter:

    -   datamodel(String) : Name of the Board Database 

    """
    
    query = QUERY_PATH + "/" + datamodel + "/schema/Entities"
    
    return self.__api_call_get(query)

In [ ]:
#export
    
def entity(self,datamodel: str,entity: str):
    
    """
    Return entity members for specified entity.
    
    The Function takes 2 Parameters:

    -   datamodel(String) : Name of the Board Database 
    -   entity(String) : Name of the specified entity 
    
    """
    
    query = QUERY_PATH + "/" + datamodel + "/schema/Entities/" + entity
    
    return self.__api_call_get(query)

In [ ]:
#export

def procedure_run(self,datamodel: str,procedure: str):
    
    """
    Execute a Board Procedure and returns Session ID of the execution.

    The Function takes 2 Parameters:

    -   datamodel(String) : Name of the Board Database 
    -   procedure(String) : Board Procedure 
    
    """
    
    query = QUERY_PATH + "/" + datamodel + "/procedure/Execute/" + procedure
    
    return self.__api_call_post(query)

In [ ]:
#export 

def procedure_status(self,datamodel: str ,session: str):
    
    """
    Return status of the execution of a Board Procedure
    
    The Function takes 2 Parameters:

    -   datamodel(String) : Name of the Board Database 
    -   session(String) : Session
    
    """
    
    query = QUERY_PATH + "/" + datamodel + "/procedure/Status/" + session
    
    return self.__api_call_get(query)